# Preprocesado de datos

In [1]:
import json
import numpy as np
import csv 
import sys
import locale
locale.setlocale(locale.LC_ALL, 'en_US')

#Data from: Instuto Nacional de Estadística www.ine.es
f = open("./sources/viajeros.txt", "r")
reader = csv.reader(f)

max_value = -1

names = []
viaj2000_list = []
viaj2016_list = []

for row in reader:
    tokens=row[0].split(";")
    name=tokens[0]
    if name in ["Eslovaquia", "Hungría", "Islandia"]:
        continue
    names.append(name)
    tokens[1]=tokens[1].split(".")[0]
    tokens[2]=tokens[2].split(".")[0]
    
    viaj2000=int(tokens[1])
    if(max_value < viaj2000):
        max_value = viaj2000
    viaj2000_list.append(viaj2000)
    viaj2016=int(tokens[2])
    if(max_value < viaj2016):
        max_value = viaj2016
    
    viaj2016_list.append(viaj2016)
    
f.close()



# Creando fuente de datos

In [2]:
from bokeh.models import ColumnDataSource

rang_list = list(range(0, len(names[1:])))

rang1 = [x + 0.3 for x in rang_list]
rang2 = [x + 0.8 for x in rang_list]

total_2000 = np.sum(viaj2000_list[1:])
total_2016 = np.sum(viaj2016_list[1:])

perc_viaj2000_list = [x * 100.0/total_2000  if x != 0 else 0 for x in viaj2000_list[1:]]
perc_viaj2016_list = [x * 100.0/total_2016   if x != 0 else 0 for x in viaj2016_list[1:]]


viaj2000ListLocale= [locale.format('%d', x, grouping=True)  for x in viaj2000_list[1:]]
viaj2016ListLocale= [locale.format('%d', x, grouping=True)  for x in viaj2016_list[1:]]


source1 = ColumnDataSource(
    data=dict(
        namesAtt=names[1:],
        v2000Att=viaj2000ListLocale,
        v2016Att=viaj2016ListLocale,
        v2000AttPerc=perc_viaj2000_list,
        v2016AttPerc=perc_viaj2016_list,
        rang1att=rang1, 
        rang2att=rang2, 
    ),
    column_names=["Name", "Travelers_2000", "Travelers_2016",  "Travelers_2000_perc",  "Travelers_2016_perc", "rang1", "rang2s"]
    
)

# Visualización de datos

In [5]:
import numpy
from bokeh.palettes import PuBu
from bokeh.io import show, output_notebook, output_file
from bokeh.models import ColumnDataSource, ranges, HoverTool
from bokeh.plotting import figure

output_notebook()

x_label = "Países"
y_label = "Visitantes de otros países (%)"
title = "Visitantes por país de residencia (2000 vs 2016)"
plot = figure(plot_width=700, plot_height=400, tools="save",
        x_axis_label = x_label,
        y_axis_label = y_label,
        title=title,
        x_range = names[1:])

bar1 = plot.vbar(source=source1, x='rang1att', top='v2000AttPerc', bottom=0,width=0.4,color='#AE9E59', legend='2000')
bar2 = plot.vbar(source=source1, x='rang2att', top='v2016AttPerc', bottom=0,width=0.4,color='#4F4478', legend='2016')

plot.xaxis.major_label_orientation = 45
plot.add_tools(HoverTool(renderers=[bar1], tooltips=[("Year", "2000"), ("Porcentaje: ", "@v2000AttPerc"), ("Núm.Personas : ", "@v2000Att")]))
plot.add_tools(HoverTool(renderers=[bar2], tooltips=[("Year", "2016"), ("Porcentaje: ", "@v2016AttPerc"), ("Núm.Personas : ", "@v2016Att")]))

output_file("../../../site/Travelings.html", title="Viajeros entrados por país de residencia")
#plot.add_layout(labels)
show(plot)

Loading BokehJS ...